20class分類を行う．

# 1. 基本設定

In [1]:
import sys
sys.path.append('/home/is/akiyoshi-n/my-project')

# 2. 使用ライブラリ

In [2]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_multiclass_dataset, split_test_data, load_text_dataset, split_multilabel_data
from src.my_project.train_v2 import MultiClassClassifier
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np

In [3]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

### パラメータの設定

In [4]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 5
# 早期停止のための忍耐値
PATIENCE = 3
# 乱数シード
SEED = 2024
# 閾値
THRESH = 0.5

In [5]:
# データの読み込み
data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")

In [7]:
class_name

array(['1 - A. 睡眠', '1 - B. 身の回りの用事', '1 - C - a. 食事-飲酒あり',
       '1 - C - b. 食事-飲酒なし・不明', '2 - A - a. 通勤・通学', '2 - A - b. その他の移動',
       '2 - B. 仕事', '2 - C. 学業', '2 - D. 家事', '2 - E. 介護・看護', '2 - F. 育児',
       '3 - A. メディア', '3 - B. 買い物', '3 - C - a. 趣味・娯楽-体動かす',
       '3 - C - b. 趣味・娯楽-体動かさない', '3 - D - a. 交際-物理的接触',
       '3 - D - b. 交際-オンライン的接触', '3 - E. 受診・療養', '3 - F - b. その他の活動',
       '4. 行動なし'], dtype=object)

In [8]:
data_labels_np = np.array(data['labels'])
data_labels_np.sum(axis=0)

array([  7,  13,  10,  32,   8,  47,  40,   2,  37,   9,  19,  57,  65,
        45,  61,  52,  21,  40,  40, 467])

In [9]:
# クラス数
NUM_LABELS = len(data['labels'][0])
NUM_LABELS

20

In [8]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = MultiClassClassifier(model_name = MODEL_NAME, num_labels=NUM_LABELS, seed=SEED, thresh=THRESH)

In [9]:
# 訓練データと評価データを辞書型で抽出
train_dataset = {
    'texts': [data['texts'][i] for i in range(900)],
    'labels': [data['labels'][i] for i in range(900)]
}
eval_dataset = {
    'texts': [data['texts'][i] for i in range(900, 1100)],
    'labels': [data['labels'][i] for i in range(900, 1100)]
}

In [10]:
trainer = Classifier_model.train_model(train_dataset, eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='21classMultiClassification', run_name='test')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.643300,0.565214,0.060000,0.045160
2,0.477800,0.353875,0.640000,0.039024
3,0.304400,0.227531,0.640000,0.039024
4,0.219600,0.177154,0.640000,0.039024
5,0.186600,0.156126,0.640000,0.039024
6,0.170200,0.143990,0.640000,0.039024
7,0.161600,0.136145,0.640000,0.039144
8,0.155000,0.130443,0.640000,0.039144
9,0.150300,0.126370,0.570000,0.038000
10,0.146500,0.122444,0.535000,0.037943


KeyboardInterrupt: 

### Cross Validation

In [10]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = MultiClassClassifier(model_name=MODEL_NAME, num_labels=NUM_LABELS, seed=SEED, thresh=THRESH)

In [11]:
# testデータと訓練に使用するデータに分割
dataset, test_data, train_index, test_index = split_test_data(data=data, test_size=0.2, SEED=SEED)

In [13]:
print(np.array(dataset['labels']).sum(axis=0))
print(np.array(test_data['labels']).sum(axis=0))

[  5  10   7  28   7  39  32   1  26   7  15  46  50  38  48  45  18  29
  33 374]
[ 2  3  3  4  1  8  8  1 11  2  4 11 15  7 13  7  3 11  7 93]


In [14]:
result = Classifier_model.cross_validation(dataset, test_data, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, NUM_FOLDS, output_dir, project_name='21Classification_cross_validation')

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/637 [00:00<?, ? examples/s]

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.659100,0.507564,0.000000,0.024484,"[0.0, 0.0, 0.0, 0.041, 0.0, 0.0, 0.083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.088, 0.0, 0.0, 0.0, 0.154, 0.0, 0.0, 0.123]","[0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.167, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.068]","[0.0, 0.0, 0.0, 0.022, 0.0, 0.0, 0.056, 0.0, 0.0, 0.0, 0.0, 0.0, 0.052, 0.0, 0.0, 0.0, 0.111, 0.0, 0.0, 0.714]"
2,0.340000,0.200767,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.168200,0.135338,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0.129400,0.115582,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,0.115700,0.107852,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.109600,0.104025,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,0.104200,0.102022,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,0.102200,0.100419,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,0.101700,0.098712,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
10,0.095400,0.095252,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


...

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.620200,0.498679,0.185185,0.030883,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.108, 0.51]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.286, 0.533]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.067, 0.488]"
2,0.356000,0.214937,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.177000,0.137871,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0.128500,0.116569,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,0.114500,0.108719,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.109300,0.104746,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,0.104800,0.102343,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,0.103900,0.101018,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,0.100400,0.099161,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
10,0.098200,0.096065,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

{'accuracy': 0.7468354430379747, 'macro_f1': 0.43952380952380954, 'class_f1': [0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 0.8, 0.0, 0.333, 0.0, 0.0, 1.0, 0.8, 1.0, 1.0, 0.857, 0.0, 1.0, 0.4, 0.85], 'class_recall': [0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 0.667, 0.0, 0.2, 0.0, 0.0, 1.0, 0.857, 1.0, 1.0, 0.75, 0.0, 1.0, 0.25, 0.944], 'class_precision': [0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.75, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.773]}


In [16]:
# 重みありの場合の結果（cv=5）
average_accuracy = round(sum(d['accuracy'] for d in result)/len(result), 3)
average_macro_f1 = round(sum(d['macro_f1'] for d in result)/len(result), 3)
# クラスごとの平均値を計算
average_class_f1 = [round(sum(d['class_f1'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_f1']))]
average_class_recall = [round(sum(d['class_recall'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_recall']))]
average_class_precision = [round(sum(d['class_precision'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_precision']))]
print("Average accuracy:", average_accuracy)
print("Average Macro f1:", average_macro_f1)
print("Average Class f1:", average_class_f1)
print("Average Class recall:", average_class_recall)
print("Average Class precision:", average_class_precision)

Average accuracy: 0.744
Average Macro f1: 0.421
Average Class f1: [0.0, 0.0, 0.0, 0.133, 0.0, 0.715, 0.798, 0.0, 0.371, 0.0, 0.0, 0.873, 0.787, 0.932, 0.88, 0.76, 0.133, 0.871, 0.304, 0.86]
Average Class recall: [0.0, 0.0, 0.0, 0.1, 0.0, 0.697, 0.767, 0.0, 0.27, 0.0, 0.0, 0.844, 0.783, 1.0, 0.876, 0.67, 0.1, 0.9, 0.217, 0.92]
Average Class precision: [0.0, 0.0, 0.0, 0.2, 0.0, 0.767, 0.883, 0.0, 0.667, 0.0, 0.0, 0.909, 0.795, 0.887, 0.888, 0.9, 0.2, 0.86, 0.567, 0.81]


In [17]:
# 重みありの場合の結果（cv=5）
average_accuracy = round(sum(d['accuracy'] for d in result)/len(result), 3)
average_macro_f1 = round(sum(d['macro_f1'] for d in result)/len(result), 3)
# クラスごとの平均値を計算
average_class_f1 = [round(sum(d['class_f1'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_f1']))]
average_class_recall = [round(sum(d['class_recall'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_recall']))]
average_class_precision = [round(sum(d['class_precision'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_precision']))]
print("Average accuracy:", average_accuracy)
print("Average Macro f1:", average_macro_f1)
print("Average Class f1:", average_class_f1)
print("Average Class recall:", average_class_recall)
print("Average Class precision:", average_class_precision)

Average accuracy: 0.596
Average Macro f1: 0.332
Average Class f1: [0.0, 0.0, 0.0, 0.189, 0.0, 0.08, 0.905, 0.0, 0.467, 0.2, 0.0, 0.966, 0.871, 0.737, 0.28, 0.698, 0.0, 0.903, 0.124, 0.215]
Average Class recall: [0.0, 0.0, 0.0, 0.18, 0.0, 0.1, 1.0, 0.0, 0.447, 0.2, 0.0, 0.938, 1.0, 0.91, 0.267, 0.711, 0.0, 0.96, 0.09, 0.184]
Average Class precision: [0.0, 0.0, 0.0, 0.2, 0.0, 0.067, 0.85, 0.0, 0.5, 0.2, 0.0, 1.0, 0.774, 0.633, 0.35, 0.726, 0.0, 0.86, 0.2, 0.375]


### 予測用，全データを学習データにして予測

In [36]:
# データの読み込み
data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")

In [37]:
for sublist in data['labels']:
    sublist.insert(-2, 0)

In [38]:
# データの読み込み
data_add, class_name_add = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation_2023_season2.xlsx")

In [39]:
# data_addをの中から500件だけ抽出（stratifiedを用いて）
data_add_500, _ = split_multilabel_data(data=data_add, test_size=0.6, SEED=SEED)

In [41]:
len(data_add_500['texts'])

523

In [35]:
len(data_add['texts'])

1284

In [42]:
# testデータと訓練に使用するデータに分割
dataset, eval_data = split_multilabel_data(data=data, test_size=0.2, SEED=SEED)

In [43]:
print(np.array(dataset['labels']).sum(axis=0))
print(np.array(eval_data['labels']).sum(axis=0))
print(np.array(data_add_500['labels']).sum(axis=0))

[  6  10   8  26   6  38  32   2  30   7  15  46  52  36  49  42  17  32
   0  32 374]
[ 1  3  2  6  2  9  8  0  7  2  4 11 13  9 12 10  4  8  0  8 93]
[  4   9   6  18   3  52  27   4  18  14  23  32  37  26  31  47  14  28
   2  16 205]


In [44]:
NUM_LABELS = 21

In [45]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = MultiClassClassifier(model_name=MODEL_NAME, num_labels=NUM_LABELS, seed=SEED, thresh=THRESH)

In [46]:
# 学習を行う
trainer = Classifier_model.train_model_adding_data(dataset, eval_data, data_add_500, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.577500,0.352243,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,0.236700,0.147608,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.139000,0.103247,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0.114200,0.090487,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,0.104600,0.086409,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.102200,0.083856,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,0.097700,0.079325,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,0.090100,0.073164,0.004902,0.007937,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.167, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.091, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,0.080000,0.065689,0.044118,0.047985,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.308, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.182, 0.538, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
10,0.068700,0.062468,0.132353,0.178219,"[0.0, 0.0, 0.0, 0.25, 0.0, 0.6, 0.222, 0.0, 0.0, 0.0, 0.0, 0.588, 0.783, 0.364, 0.154, 0.167, 0.0, 0.615, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.167, 0.0, 0.667, 0.125, 0.0, 0.0, 0.0, 0.0, 0.455, 0.692, 0.222, 0.083, 0.1, 0.0, 0.5, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.5, 0.0, 0.545, 1.0, 0.0, 0.0, 0.0, 0.0, 0.833, 0.9, 1.0, 1.0, 0.5, 0.0, 0.8, 0.0, 0.0, 0.0]"


### Cross Validation add data

In [18]:
# データの読み込み
data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")

In [19]:
for sublist in data['labels']:
    sublist.insert(-2, 0)

In [20]:
len(data['labels'][0])

21

In [21]:
len(class_name)

20

In [22]:
# データの読み込み
data_add, class_name_add = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation_2023_season2.xlsx")

In [23]:
len(class_name_add)

21

In [24]:
# data_addをの中から500件だけ抽出（stratifiedを用いて）
data_add_500, _ = split_multilabel_data(data=data_add, test_size=0.6, SEED=SEED)

In [26]:
len(data_add_500['labels'])

523

In [27]:
print(np.array(data_add_500['labels']).sum(axis=0))

[  4   9   6  18   3  52  27   4  18  14  23  32  37  26  31  47  14  28
   2  16 205]


In [24]:
NUM_LABELS = 21

In [25]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = MultiClassClassifier(model_name=MODEL_NAME, num_labels=NUM_LABELS, seed=SEED, thresh=THRESH)

In [26]:
# testデータと訓練に使用するデータに分割
dataset, test_data, train_index, test_index = split_test_data(data=data, test_size=0.2, SEED=SEED)

In [27]:
# testデータと訓練に使用するデータに分割
dataset, test_data = split_multilabel_data(data=data, test_size=0.2, SEED=SEED)

In [28]:
print(np.array(dataset['labels']).sum(axis=0))
print(np.array(test_data['labels']).sum(axis=0))

[  6  10   8  26   6  38  32   2  30   7  15  46  52  36  49  42  17  32
   0  32 374]
[ 1  3  2  6  2  9  8  0  7  2  4 11 13  9 12 10  4  8  0  8 93]


In [29]:
print(np.array(dataset['labels']).sum(axis=0))
print(np.array(test_data['labels']).sum(axis=0))

[  6  10   8  26   6  38  32   2  30   7  15  46  52  36  49  42  17  32
   0  32 374]
[ 1  3  2  6  2  9  8  0  7  2  4 11 13  9 12 10  4  8  0  8 93]


In [30]:
result = Classifier_model.cross_validation_add_dataset(dataset, data_add_500, test_data, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, NUM_FOLDS, output_dir, project_name='21Classification_cross_validation')

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1165 [00:00<?, ? examples/s]

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

Map:   0%|          | 0/161 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.591000,0.384011,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,0.259300,0.159865,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.145600,0.109205,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0.116100,0.094194,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,0.108000,0.089582,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.106300,0.086855,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,0.102800,0.082970,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,0.093700,0.077599,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,0.086000,0.072904,0.062112,0.063910,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.842, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.727, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
10,0.076800,0.068985,0.099379,0.164928,"[0.0, 0.0, 0.0, 0.333, 0.0, 0.6, 0.4, 0.0, 0.0, 0.0, 0.0, 0.667, 0.778, 0.0, 0.0, 0.4, 0.0, 0.286, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.2, 0.0, 0.429, 0.286, 0.0, 0.0, 0.0, 0.0, 0.556, 0.636, 0.0, 0.0, 0.25, 0.0, 0.167, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.833, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]"


Map:   0%|          | 0/204 [00:00<?, ? examples/s]

{'accuracy': 0.8188976377952756, 'macro_f1': 0.49475781536531976, 'class_f1': [0.0, 0.0, 0.0, 0.667, 0.0, 1.0, 0.444, 0.0, 0.0, 0.667, 1.0, 0.769, 0.87, 1.0, 0.769, 0.0, 0.8, 1.0, 0.0, 0.5, 0.904], 'class_recall': [0.0, 0.0, 0.0, 0.667, 0.0, 1.0, 0.333, 0.0, 0.0, 1.0, 1.0, 0.625, 0.833, 1.0, 0.625, 0.0, 0.667, 1.0, 0.0, 0.333, 0.971], 'class_precision': [0.0, 0.0, 0.0, 0.667, 0.0, 1.0, 0.667, 0.0, 0.0, 0.5, 1.0, 1.0, 0.909, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.846]}


In [31]:
# 重みありの場合の結果（cv=5）
average_accuracy = round(sum(d['accuracy'] for d in result)/len(result), 3)
average_macro_f1 = round(sum(d['macro_f1'] for d in result)/len(result), 3)
# クラスごとの平均値を計算
average_class_f1 = [round(sum(d['class_f1'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_f1']))]
average_class_recall = [round(sum(d['class_recall'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_recall']))]
average_class_precision = [round(sum(d['class_precision'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_precision']))]
print("Average accuracy:", average_accuracy)
print("Average Macro f1:", average_macro_f1)
print("Average Class f1:", average_class_f1)
print("Average Class recall:", average_class_recall)
print("Average Class precision:", average_class_precision)

Average accuracy: 0.756
Average Macro f1: 0.469
Average Class f1: [0.0, 0.133, 0.0, 0.583, 0.0, 0.857, 0.495, 0.0, 0.13, 0.66, 0.683, 0.799, 0.861, 0.762, 0.635, 0.519, 0.653, 0.861, 0.0, 0.36, 0.853]
Average Class recall: [0.0, 0.1, 0.0, 0.513, 0.0, 0.975, 0.53, 0.0, 0.117, 1.0, 0.8, 0.71, 0.867, 0.653, 0.503, 0.554, 0.567, 0.971, 0.0, 0.297, 0.864]
Average Class precision: [0.0, 0.2, 0.0, 0.717, 0.0, 0.774, 0.545, 0.0, 0.15, 0.5, 0.62, 0.921, 0.86, 0.95, 0.877, 0.493, 0.8, 0.789, 0.0, 0.5, 0.846]


In [18]:
# 重みありの場合の結果（cv=5）
average_accuracy = round(sum(d['accuracy'] for d in result)/len(result), 3)
average_macro_f1 = round(sum(d['macro_f1'] for d in result)/len(result), 3)
# クラスごとの平均値を計算
average_class_f1 = [round(sum(d['class_f1'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_f1']))]
average_class_recall = [round(sum(d['class_recall'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_recall']))]
average_class_precision = [round(sum(d['class_precision'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_precision']))]
print("Average accuracy:", average_accuracy)
print("Average Macro f1:", average_macro_f1)
print("Average Class f1:", average_class_f1)
print("Average Class recall:", average_class_recall)
print("Average Class precision:", average_class_precision)

Average accuracy: 0.642
Average Macro f1: 0.341
Average Class f1: [0.0, 0.0, 0.0, 0.405, 0.0, 0.704, 0.652, 0.0, 0.2, 0.467, 0.0, 0.919, 0.867, 0.548, 0.233, 0.596, 0.0, 0.863, 0.0, 0.08, 0.626]
Average Class recall: [0.0, 0.0, 0.0, 0.363, 0.0, 0.73, 0.843, 0.0, 0.15, 0.6, 0.0, 0.908, 1.0, 0.533, 0.213, 0.633, 0.0, 0.831, 0.0, 0.067, 0.603]
Average Class precision: [0.0, 0.0, 0.0, 0.467, 0.0, 0.768, 0.55, 0.0, 0.3, 0.4, 0.0, 0.933, 0.768, 0.603, 0.267, 0.592, 0.0, 0.917, 0.0, 0.1, 0.651]


In [17]:
# 重みありの場合の結果（cv=5）
average_accuracy = round(sum(d['accuracy'] for d in result)/len(result), 3)
average_macro_f1 = round(sum(d['macro_f1'] for d in result)/len(result), 3)
# クラスごとの平均値を計算
average_class_f1 = [round(sum(d['class_f1'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_f1']))]
average_class_recall = [round(sum(d['class_recall'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_recall']))]
average_class_precision = [round(sum(d['class_precision'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_precision']))]
print("Average accuracy:", average_accuracy)
print("Average Macro f1:", average_macro_f1)
print("Average Class f1:", average_class_f1)
print("Average Class recall:", average_class_recall)
print("Average Class precision:", average_class_precision)

Average accuracy: 0.636
Average Macro f1: 0.367
Average Class f1: [0.0, 0.0, 0.0, 0.423, 0.0, 0.593, 0.618, 0.0, 0.35, 0.54, 0.0, 0.969, 0.874, 0.423, 0.45, 0.592, 0.0, 0.913, 0.0, 0.204, 0.748]
Average Class recall: [0.0, 0.0, 0.0, 0.413, 0.0, 0.663, 0.893, 0.0, 0.27, 1.0, 0.0, 0.96, 0.973, 0.329, 0.42, 0.576, 0.0, 0.841, 0.0, 0.147, 0.713]
Average Class precision: [0.0, 0.0, 0.0, 0.464, 0.0, 0.54, 0.49, 0.0, 0.533, 0.39, 0.0, 0.983, 0.801, 0.667, 0.5, 0.611, 0.0, 1.0, 0.0, 0.4, 0.811]


### Majority Baseline

In [12]:
print(np.array(dataset['labels']).sum(axis=0))

[  5  10   7  28   7  39  32   1  26   7  15  46  50  38  48  45  18  29
  33 374]


In [13]:
# テストサイズ×マルチラベル数の二次元リスト
majority_pred = [[0 for i in range(20)] for j in range(len(test_data['labels']))]
for i in range(len(test_data['labels'])):
    majority_pred[i][-1] = 1

In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [16]:
# accuracyを計算
accuracy = round(accuracy_score(y_true=test_data['labels'], y_pred=majority_pred), 3)
# macro f1を計算
macro_f1 = round(f1_score(y_true=test_data['labels'], y_pred=majority_pred, average='macro', zero_division=0), 3)
# クラス毎のF1値を計算
class_f1 = [round(score, 3) for score in f1_score(y_true=test_data['labels'], y_pred=majority_pred, average=None, zero_division=0)]
# クラス毎のrecallを計算
class_recall = [round(score, 3) for score in recall_score(y_true=test_data['labels'], y_pred=majority_pred, average=None, zero_division=0)]
# クラス毎のprecisionを計算
class_precision = [round(score, 3) for score in precision_score(y_true=test_data['labels'], y_pred=majority_pred, average=None, zero_division=0)]
print("Average accuracy:", accuracy)
print("Average Macro f1:", macro_f1)
print("Average Class f1:", class_f1)
print("Average Class recall:", class_recall)
print("Average Class precision:", class_precision)

Average accuracy: 0.46
Average Macro f1: 0.032
Average Class f1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.635]
Average Class recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
Average Class precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.465]
